# **Spatial Transcriptomics Deep Learning (STDL) Project Notebook**

> The notebook contains main experiments and examples of how to use the code

## **Phase 1: Pre-processing and technical preparations**

### 1.1: **Assign GPU device and allow CUDA debugging**

In [1]:
# the next 2 lines are to allow debugging with CUDA !
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  
print(f'cuda debugging allowed')

cuda debugging allowed


In [2]:
%%time

import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()

cuda device count: 1
Using device: cuda
device name: GeForce GTX 1080 Ti
torch.cuda.device(0): <torch.cuda.device object at 0x7feb603a9c90>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
CPU times: user 1.47 s, sys: 4.68 s, total: 6.14 s
Wall time: 13.9 s


### 1.2: **Import some of the project modules (more will be loaded later)**

> `loadAndPreProcess` module contains methods to load the data files as pytorch and pandas objects, methods to preprocess the given data, and methods to create custom datasets from the preprocessed data.

> `deepNetworkArchitechture` module contains .... write this :sad:  :\

<div class="alert alert-block alert-warning">
<b>TODO:</b> fill above line
</div>

In [ ]:
# create code to reimport module if i change it
%load_ext autoreload
%autoreload 2

# note: path to project is: /home/roy.rubin/STDLproject/
import loadAndPreProcess
import deepNetworkArchitechture

### 1.3: **Load pytorch dataset objects from the image folder**

> Note that `augmentedImageFolder` is a custom dataset of imageFolder objects with different transformations (see code).

In [ ]:
%%time

path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
imageFolder = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir)
augmentedImageFolder = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir)

### 1.4: **Load pandas dataframe objects from the 3 given tsv/csv files**

> `matrix_dataframe` represents the gene expression count values of each sample for each gene

> `features_dataframe` contains the names of all the genes

> `barcodes_datafame` contains the names of all the samples

In [ ]:
%%time

path_to_mtx_tsv_files_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData"
matrix_dataframe, features_dataframe, barcodes_datafame = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir)

### 1.5: **Remove less-informative genes**

> we define *less-informative* genes as genes with less than K counts over all samples

> K is a parameter for the user's choice

In [ ]:
%%time

Base_value = 10
matrix_dataframe, mapping_between_old_and_new_indices = loadAndPreProcess.cut_genes_with_under_B_counts(matrix_dataframe, Base_value) 
# # uncomment if wanted:
# print(f'\nnote: this is the mapping_between_old_and_new_indices: \n{mapping_between_old_and_new_indices}')

### 1.6: **Normalize matrix_dataframe entries**

> normaliztion will be performed on the remainning rows of the dataframe with the logic "log 1P"

> This method Calculates log(1 + x)

In [ ]:
%%time

matrix_dataframe = loadAndPreProcess.perform_log_1p_normalization(matrix_dataframe) 

### 1.7: **Create custom datasets**

> Each custom dataset is tailored per task

> there are four tasks: single gene prediction, k gene prediction, all gene prediction using NMF dim. reduction, all gene prediction using AE dim. reduction

> For each of the above tasks 2 datasets were created - one with the regular images, and one with the augmented dataset - images with transformations.


In [ ]:
%%time
gene_name = 'MKI67'
custom_DS_SingleValuePerImg = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name,
                                                               index_mapping=mapping_between_old_and_new_indices)
custom_DS_SingleValuePerImg_augmented = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=augmentedImageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name,
                                                               index_mapping=mapping_between_old_and_new_indices)

<div class="alert alert-block alert-info">
<b>Note:</b> inside the init phase of `STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance` class, K genes with the highest variance are chosen from matrix_dataframe, and they are the only genes that are kept for training and testing purposes
</div>

In [ ]:
%%time

k = 10
custom_DS_KGenesWithHighestVariance = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)

<div class="alert alert-block alert-info">
<b>Note:</b> inside the init phase of `STDL_Dataset_KValuesPerImg_LatentTensor_NMF` class, an NMF decompositionis performed on the matrix_dataframe object
</div>

In [ ]:
%%time

k = 10
custom_DS_LatentTensor_NMF = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)

<div class="alert alert-block alert-info">
<b>Note:</b> inside the init phase of `custom_DS_LatentTensor_AE` class, an Autoencoder network is being trained.
</div>

In [ ]:
%%time

k = 10
custom_DS_LatentTensor_AE = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)

### 1.8: prepare for the next phases in which the experiments are executed

> import `executionModule` which contains the experiments, training methods, and testing methods

> create `hyperparameters` dictionary which will contain all of the hyper-parameters for our experiments (note - user can change these later)

> create `experiment_model_list` that will hold all the experiment methods in which the stated model is used for the experiment's execution

<div class="alert alert-block alert-warning">
<b>Warning:</b> change the hyper-parameters below with caution if needed !
</div>

In [98]:
%%time

import executionModule

# define hyperparameters for the experiments
hyperparameters = dict()
hyperparameters['batch_size'] = 25
hyperparameters['max_alowed_number_of_batches'] = 99999
hyperparameters['precent_of_dataset_allocated_for_training'] = 0.8
hyperparameters['learning_rate'] = 1e-4
hyperparameters['momentum'] = 0.9
hyperparameters['num_of_epochs'] = 3
hyperparameters['channels'] = [32] 
hyperparameters['num_of_convolution_layers'] = len(hyperparameters['channels'])
hyperparameters['hidden_dims'] = [100]
hyperparameters['num_of_hidden_layers'] = len(hyperparameters['hidden_dims'])
hyperparameters['pool_every'] = 99999

experiment_model_list = []
experiment_model_list.append(executionModule.runExperimentWithModel_BasicConvNet)
experiment_model_list.append(executionModule.runExperimentWithModel_DenseNet121)
# experiment_model_list.append(executionModule.runExperimentWithModel_InceptionV3) #TODO: still need to sort out input image size... !

CPU times: user 47 µs, sys: 1e+03 ns, total: 48 µs
Wall time: 58.4 µs


<div class="alert alert-block alert-danger">
<b>Note:</b> inception_v3 model isnt sorted out yet... gives
    RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (5 x 5). Kernel size can't be greater than actual input size
</div>

## Phase 2: Single Gene Prediction

In [99]:
for index,exp in enumerate(experiment_model_list):
    print(f'\nstarting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_SingleValuePerImg, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}')

starting experiment 0 with DS: custom_DS_SingleValuePerImg_augmented


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
****** begin training ******

iteration 1 of 3 epochs
batch 122 of 122 batches
finished inner loop.
in this epoch: average loss 3.088161685336076

iteration 2 of 3 epochs
batch 122 of 122 batches
finished inner loop.
in this epoch: average loss 0.368179158597696

iteration 3 of 3 epochs
batch 122 of 122 batches
finished inner loop.
in this epoch: average loss 0.36589690664264024
finished all epochs !
 \ * / FINISHED train_prediction_model \ * / 

----- finished function runExperimentWithModel_BasicConvNet -----
CPU times: user 20min 36s, sys: 7.91 s, total: 20min 44s
Wall time: 1min 38s

finished experiment 0

starting experiment 1 with DS: custom_DS_SingleValuePerImg_augmented


----- entered function runExperimentWithModel_DenseNet121 -----
/ * \ ENTERED train_prediction_model / * \ 
****** begin training *

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_SingleValuePerImg_augmented, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

## Phase 3: K genes prediction

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_KGenesWithHighestVariance, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_KGenesWithHighestVariance_augmented, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

## Phase 4: All genes prediction - using dimensionality reduction techniques

### 4.1: Prediction using dimensionality reduction technique NMF

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_LatentTensor_NMF, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_LatentTensor_NMF_augmented, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

### 4.1: Prediction using dimensionality reduction technique AE

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_LatentTensor_AE, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

In [ ]:
for index,exp in enumerate(experiment_model_list):
    print(f'starting experiment {index+1} with DS: custom_DS_SingleValuePerImg_augmented\n')
    %time exp(custom_DS_LatentTensor_AE_augmented, hyperparams=hyperparameters, device=device, dataset_name='single_gene')
    print(f'\nfinished experiment {index+1}\n')

<div class="alert alert-block alert-danger">
<b>Note:</b> not tested yet
</div>

In [96]:
%%timeit -n 1 -r 1 # dont try to run multiple times to time it, run only once

executionModule.runExperimentWithModel_InceptionV3(custom_DS_LatentTensor_AE_augmented, hyperparams=hyperparameters, device=device, dataset_name='AE')


----- entered function runExperimentWithModel_InceptionV3 -----
starting to load the model inception_v3 from torchvision.models. this is quite heavy, and might take some time ...
finished loading model
/ * \ ENTERED train_prediction_model / * \ 
****** begin training ******

iteration 1 of 3 epochs


RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (5 x 5). Kernel size can't be greater than actual input size